In [16]:
import nibabel as nib
import numpy as np
import os
import datalad.api

def split_nii_file(file_path,output_folder, chunk_size=15):
    # Check if file is a symbolic link
    # The last 4 elements of the path are dataset/sub-*/func/*.nii.gz
    # I need to set DataLad to be run from the dataset folder
    # The file path thet datalad uses should be the last 3 elements
    if os.path.islink(file_path):
        # Use datalad to get the real file path
        datalad_file_path = file_path.split('/')[-3:]
        datalad_file_path = '/'.join(datalad_file_path)
        datalad.api.get(datalad_file_path, result_xfm='paths', return_type='item-or-list')
        

    img = nib.load(file_path)
    data = img.get_fdata()
    data = data[:,:,:,:180]
    
    # Determine the number of chunks
    total_time_points = data.shape[-1]
    num_chunks = total_time_points // chunk_size
    
    # Split the data into chunks
    chunks = np.array_split(data, num_chunks, axis=-1)
    
    # Save each chunk as a new NIfTI file (optional)
    for i, chunk in enumerate(chunks):
        chunk_img = nib.Nifti1Image(chunk, img.affine, img.header)
        nib.save(chunk_img, f"../{output_folder}/{os.path.basename(file_path).replace('.nii.gz', f'_chunk{i}.nii.gz')}")
    

def split_nii_files_dataset(file_suffix, chunk_size, output_folder):
    # Create output folder if it does not exist
    if not os.path.exists("../"+output_folder):
        os.makedirs("../"+output_folder)
    # List folders in the dataset
    
    folders = os.listdir(".")
    
    
    # Useful folders are those that start with 'sub-'
    useful_folders = [folder for folder in folders if folder.startswith('sub-') and os.path.isdir(os.path.abspath(folder))]
    
    for folder in useful_folders:
        # List files in {folder}/func ending in file_suffix
        func_folder = os.path.join(folder, 'func')
        files = os.listdir(func_folder)
        useful_files = [file for file in files if file.endswith(file_suffix + '.nii.gz')]
        useful_files = [file for file in useful_files if "emotionalfaces_run" in file]
        for file in useful_files:
            file_path = os.path.join(func_folder, file)
            split_nii_file(file_path, output_folder, chunk_size)


In [17]:

# Run datalad install https://github.com/OpenNeuroDerivatives/ds003548-fmriprep.git before running this script
# If current folder is not the dataset folder, change to it
if not 'ds003548-fmriprep' == os.getcwd().split('/')[-1]:
    os.chdir('ds003548-fmriprep')
split_nii_files_dataset('Asym_res-2_desc-preproc_bold', 15,'ml-chunks')

[WARNING] Failed to get 'sourcedata/templateflow/tpl-Fischer344.path', skipping this submodule 
[WARNING] Failed to get 'sourcedata/templateflow/tpl-fsaverage.path', skipping this submodule 
[WARNING] Failed to get 'sourcedata/templateflow/tpl-fsLR.path', skipping this submodule 
[WARNING] Failed to get 'sourcedata/templateflow/tpl-MNI152Lin.path', skipping this submodule 
[WARNING] Failed to get 'sourcedata/templateflow/tpl-MNI152NLin2009aAsym.path', skipping this submodule 
[WARNING] Failed to get 'sourcedata/templateflow/tpl-MNI152NLin2009aSym.path', skipping this submodule 
[WARNING] Failed to get 'sourcedata/templateflow/tpl-MNI152NLin2009bAsym.path', skipping this submodule 
[WARNING] Failed to get 'sourcedata/templateflow/tpl-MNI152NLin2009bSym.path', skipping this submodule 
[WARNING] Failed to get 'sourcedata/templateflow/tpl-MNI152NLin2009cAsym.path', skipping this submodule 
[WARNING] Failed to get 'sourcedata/templateflow/tpl-MNI152NLin2009cSym.path', skipping this submodul

get(ok): sub-06/func/sub-06_task-emotionalfaces_run-3_space-MNI152NLin2009cAsym_res-2_desc-brain_mask.nii.gz (file) [from openneuro-derivatives...]


IndexError: too many indices for array: array is 3-dimensional, but 4 were indexed